In [ ]:
# script to replace the base mechanism with new reactions (every reaction calculated) and species (all species calculated for now)

# this script makes use of the revised Disproportion training family

In [1]:
import os
import pickle
import datetime
import subprocess
import numpy as np

import rmgpy
import rmgpy.chemkin
import rmgpy.data.kinetics


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def plot_kinetics(rxns, labels=None):
    """Function for plotting reaction kinetics
    Takes in a list of RMG reactions (rmgpy.reaction.Reaction) or a single reaction
    """
    plt.xlabel('1000 / T (K^-1)')
    plt.ylabel('ln(k)')

    if type(rxns) != list:
        rxns = [rxns]
    
    T = np.linspace(300, 3000, 1001)
    for rxn in rxns:
        k = np.zeros(len(T))
        for i in range(0, len(T)):
            k[i] = rxn.get_rate_coefficient(T[i], 101325)
        plt.plot(1000.0 / T, np.log(k))

    if labels:
        plt.legend(labels)
    plt.show()

In [3]:
def plot_thermos(thermos, labels=None):
    %matplotlib inline
    fig, ax = plt.subplots(1,3)
    fig.set_size_inches(12, 3)
    fig.tight_layout()
    ax[0].set_xlabel('Temperature (K)')
    ax[0].set_ylabel('H (kJ / mol)')
    ax[0].set_title('Enthalpy vs. Temperature')
    ax[1].set_xlabel('Temperature (K)')
    ax[1].set_ylabel('S (kJ / mol K)')
    ax[1].set_title('Entropy vs. Temperature')
    ax[2].set_xlabel('Temperature (K)')
    ax[2].set_ylabel('Cp (kJ / mol K)')
    ax[2].set_title('Heat Capacity vs. Temperature')
    T = np.linspace(300, 3000, 1001)
    for thermo in thermos:
        H = np.zeros(len(T))
        S = np.zeros(len(T))
        Cp = np.zeros(len(T))
        for i in range(0, len(T)):
            H[i] = thermo.get_enthalpy(T[i]) / 1000.0
            S[i] = thermo.get_entropy(T[i]) / 1000.0
            Cp[i] = thermo.get_heat_capacity(T[i]) / 1000.0
        ax[0].plot(T, H)
        ax[1].plot(T, S)
        ax[2].plot(T, Cp)
    ax[0].legend(labels)
    ax[1].legend(labels)
    ax[2].legend(labels)
    plt.subplots_adjust(wspace=0.25)
    plt.show()

In [4]:
# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model'
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport, use_chemkin_names=True)
print(f'{len(species_list)} species, {len(reaction_list)} reactions')


110 species, 1822 reactions


In [5]:
# load the thermo database
thermo_libs = [
    'BurkeH2O2',
    'primaryThermoLibrary',
    'FFCM1(-)',
    'CurranPentane',
    'Klippenstein_Glarborg2016',
    'thermo_DFT_CCSDTF12_BAC',
    'DFT_QCI_thermo',
    'CBS_QB3_1dHR',
]

thermo_library_path = os.path.join(rmgpy.settings['database.directory'], 'thermo')
thermo_database = rmgpy.data.thermo.ThermoDatabase()
thermo_database.load(
    thermo_library_path,
    libraries=thermo_libs
)

In [6]:
# load the revised Disproportionation family
family = 'Disproportionation'
ref_library_path = os.path.join(rmgpy.settings['database.directory'], 'kinetics')
kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
kinetics_database.load(
    ref_library_path,
    libraries=[],
    families=[family]
)


In [7]:
# load the entire database
ref_db = rmgpy.data.rmg.RMGDatabase()
ref_db.kinetics = kinetics_database
ref_db.thermo = thermo_database

In [8]:
def get_species(lib_sp):
    for mechanism_sp in species_list:
        if mechanism_sp.is_isomorphic(lib_sp):
            return mechanism_sp

In [9]:
modified_reaction_list = reaction_list[:]
for i in range(0, len(reaction_list)):
    if hasattr(reaction_list[i], 'family'):
        if reaction_list[i].family == family:
            print('replacing kinetics for', reaction_list[i])
            print(f' from { reaction_list[i].to_chemkin()}')
#             templates = rmgpy.data.rmg.database.kinetics.generate_reactions(
            templates = ref_db.kinetics.generate_reactions(
                reaction_list[i].reactants,
                products=reaction_list[i].products,
                only_families=[family]
            )
            
            
            for j in range(0, len(templates[0].reactants)):
                templates[0].reactants[j] = get_species(templates[0].reactants[j])
            for j in range(0, len(templates[0].products)):
                templates[0].products[j] = get_species(templates[0].products[j])
            
            new_kinetics = ref_db.kinetics.families[family].get_kinetics(templates[0], templates[0].template)[0][0]
            # convert to Arrhenius if ArrheniusBM
            if type(new_kinetics) == rmgpy.kinetics.arrhenius.ArrheniusBM:
                dHrxn = templates[0].get_enthalpy_of_reaction(1000)
                new_kinetics = new_kinetics.to_arrhenius(dHrxn)
            
            modified_reaction_list[i].kinetics = new_kinetics
            print(f'   to { modified_reaction_list[i].to_chemkin() }')


replacing kinetics for HO2(16) + C2H5(33) <=> H2O2(17) + C2H4(11)
 from ! Template reaction: Disproportionation
! Flux pairs: HO2(16), H2O2(17); C2H5(33), C2H4(11); 
! Estimated from node Root_N-4R->H_4CNOS-u1_N-1R!H->O_4CNOS->O_Ext-4O-R_5R!H-u0
! Multiplied by reaction path degeneracy 3.0
HO2(16)+C2H5(33)=H2O2(17)+C2H4(11)                  1.253070e+08 1.719     0.000    

   to ! Template reaction: Disproportionation
! Flux pairs: HO2(16), H2O2(17); C2H5(33), C2H4(11); 
! Fitted to 50 data points; dA = *|/ 1.15458, dn = +|- 0.0188838, dEa = +|- 0.102765 kJ/molMatched reaction 137 C2H5 + HO2-2 <=> H2O2 + C2H4 in Disproportionation/training
! This reaction matched rate rule [Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_2R!H->C]
! family: Disproportionation
HO2(16)+C2H5(33)=H2O2(17)+C2H4(11)                  1.000700e+02 3.438     11.628   

replacing kinetics for CH2(23) + C2H5(33) <=> CH3(18) + C2H4(11)
 from ! Template reaction: Disproportionation
! Flux pairs: CH2(23), CH3(18); C2H5(33), C2H

   to ! Template reaction: Disproportionation
! Flux pairs: PC4H9(182), butane(1); NC3H7(92), C3H6(12); 
! Estimated from node Root_1R!H->C_Ext-4R-R_N-4R->O_N-5R!H->S_N-Sp-5CCCCCHHNNOOOSS#4CCCCCCHHHNNNOOOSSS_Ext-1C-R_6R!H->C_Sp-6C-1C_4CHNS->C_5CO->C_Sp-5C-4C_Ext-5C-R
NC3H7(92)+PC4H9(182)=C3H6(12)+butane(1)             1.450000e+12 0.000     0.000    

replacing kinetics for PC4H9(182) + SC4H9(183) <=> C4H8(189) + butane(1)
 from ! Template reaction: Disproportionation
! Flux pairs: SC4H9(183), butane(1); PC4H9(182), C4H8(189); 
! Estimated from node Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Sp-6C-4C_Ext-6C-R
! Multiplied by reaction path degeneracy 2.0
PC4H9(182)+SC4H9(183)=C4H8(189)+butane(1)           2.900000e+12 0.000     0.000    

   to ! Template reaction: Disproportionation
! Flux pairs: SC4H9(183), butane(1); PC4H9(182), C4H8(189); 
! Estimated from node Root_1R!H->C_Ext-4R

   to ! Template reaction: Disproportionation
! Flux pairs: SC4H9(183), C4H8(188); CH2(23), CH3(18); 
! Fitted to 50 data points; dA = *|/ 1.11841, dn = +|- 0.0147024, dEa = +|- 0.0800099 kJ/molMatched reaction 139 SC4H9(183) + CH2 <=> CH3 + C4H8(188) in Disproportionation/training
! This reaction matched rate rule [Root_1R!H->C_N-2R!H->O_Ext-2CNS-R_Ext-5R!H-R_N-4R-u1_Sp-6R!H-5R!H]
! family: Disproportionation
CH2(23)+SC4H9(183)=CH3(18)+C4H8(188)                1.366110e+04 3.347     -7.539   

replacing kinetics for OH(15) + NC3H7(92) <=> H2O(8) + C3H6(12)
 from ! Template reaction: Disproportionation
! Flux pairs: OH(15), H2O(8); NC3H7(92), C3H6(12); 
! Matched reaction 33 HO + C3H7-2 <=> H2O + C3H6-2 in Disproportionation/training
! This reaction matched rate rule [Root_Ext-1R!H-R_4R->O_Sp-5R!H-1R!H]
! family: Disproportionation
OH(15)+NC3H7(92)=H2O(8)+C3H6(12)                    4.820000e+13 0.000     0.000    

   to ! Template reaction: Disproportionation
! Flux pairs: OH(15), H2

   to ! Template reaction: Disproportionation
! Flux pairs: CH2CHO(21), CH2CO(24); HCO(19), CH2O(9); 
! Estimated from node Root_1R!H->C_Ext-4R-R_N-4R->O_N-5R!H->S_N-Sp-5CCCCCHHNNOOOSS#4CCCCCCHHHNNNOOOSSS_Ext-1C-R
HCO(19)+CH2CHO(21)=CH2O(9)+CH2CO(24)                1.190500e+08 1.033     0.000    

replacing kinetics for CH2CHO(21) + CH3CO(20) <=> CH2CO(24) + CH3CHO(35)
 from ! Template reaction: Disproportionation
! Flux pairs: CH3CO(20), CH3CHO(35); CH2CHO(21), CH2CO(24); 
! Estimated from node Root_Ext-2R!H-R
! Multiplied by reaction path degeneracy 3.0
CH2CHO(21)+CH3CO(20)=CH2CO(24)+CH3CHO(35)           6.594120e+08 1.422     0.000    
DUPLICATE

   to ! Template reaction: Disproportionation
! Flux pairs: CH3CO(20), CH3CHO(35); CH2CHO(21), CH2CO(24); 
! Estimated from node Root_1R!H->C_Ext-4R-R_N-4R->O_N-5R!H->S_N-Sp-5CCCCCHHNNOOOSS#4CCCCCCHHHNNNOOOSSS_2R!H->C_5CO->C_Sp-5C-4CHNS_Ext-5C-R
CH2CHO(21)+CH3CO(20)=CH2CO(24)+CH3CHO(35)           2.290000e+13 -0.350    0.000    
DUPLICATE


   to ! Template reaction: Disproportionation
! Flux pairs: SC4H9(183), C4H8(189); CH2CHO(21), CH3CHO(35); 
! Estimated from node Root_1R!H->C_Ext-4R-R_N-4R->O_N-5R!H->S_Ext-2R!H-R_Ext-5CO-R_7R!H-u0_N-7R!H->C
CH2CHO(21)+SC4H9(183)=CH3CHO(35)+C4H8(189)          4.177230e+02 2.955     0.000    

replacing kinetics for CH3CO(20) + SC4H9(183) <=> CH3CHO(35) + C4H8(189)
 from ! Template reaction: Disproportionation
! Flux pairs: SC4H9(183), C4H8(189); CH3CO(20), CH3CHO(35); 
! Estimated from node Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C
! Multiplied by reaction path degeneracy 2.0
CH3CO(20)+SC4H9(183)=CH3CHO(35)+C4H8(189)           2.105300e+14 -0.550    0.000    

   to ! Template reaction: Disproportionation
! Flux pairs: SC4H9(183), C4H8(189); CH3CO(20), CH3CHO(35); 
! Estimated from node Root_1R!H->C_Ext-4R-R_N-4R->O_N-5R!H->S_Ext-2R!H-R
CH3CO(20)+SC4H9(183)=CH3CHO(

...

   to ! Template reaction: Disproportionation
! Flux pairs: OCHO(38), HOCHO(40); OCHO(38), CO2(7); 
! Estimated from node Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R
OCHO(38)+OCHO(38)=CO2(7)+HOCHO(40)                  1.115240e+16 -0.957    0.000    

replacing kinetics for OCHO(38) + CH3O(31) <=> CH2O(9) + HOCHO(40)
 from ! Template reaction: Disproportionation
! Flux pairs: OCHO(38), HOCHO(40); CH3O(31), CH2O(9); 
! Estimated from node Root_N-4R->H_4CNOS-u1_N-1R!H->O_4CNOS->O_Sp-2R!H-1CNS_1CNS->C_N-2R!H->C
! Multiplied by reaction path degeneracy 6.0
OCHO(38)+CH3O(31)=CH2O(9)+HOCHO(40)                 2.160000e+07 2.000     0.000    

   to ! Template reaction: Disproportionation
! Flux pairs: OCHO(38), HOCHO(40); CH3O(31), CH2O(9); 
! Estimated from node Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0
OCHO(38)+CH3O(31)=CH2O(9)+HOCHO(40)                 6.996190e+08 1.340     0.000    

replacing kinetics for OCHO(38) + C2H5(33) <=> HOCHO(40) + C2H4(11)
 from ! Template reaction: Disproportiona

   to ! Template reaction: Disproportionation
! Flux pairs: C4H8(748), SC4H9(183); OCHO(38), CO2(7); 
! Estimated from node Root_1R!H->C_Ext-4R-R_N-4R->O_N-5R!H->S_N-Sp-5CCCCCHHNNOOOSS#4CCCCCCHHHNNNOOOSSS_Ext-1C-R
OCHO(38)+C4H8(748)=CO2(7)+SC4H9(183)                1.190500e+08 1.033     0.000    

replacing kinetics for OCHO(38) + CH3O(31) <=> CO2(7) + CH3OH(30)
 from ! Template reaction: Disproportionation
! Flux pairs: OCHO(38), CO2(7); CH3O(31), CH3OH(30); 
! Estimated from node Root_Ext-1R!H-R
OCHO(38)+CH3O(31)=CO2(7)+CH3OH(30)                  9.445080e+09 0.509     0.000    

   to ! Template reaction: Disproportionation
! Flux pairs: OCHO(38), CO2(7); CH3O(31), CH3OH(30); 
! Estimated from node Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R
OCHO(38)+CH3O(31)=CO2(7)+CH3OH(30)                  1.115240e+16 -0.957    0.000    

replacing kinetics for CH3(18) + C4H8(748) <=> CH4(10) + C4H7(191)
 from ! Template reaction: Disproportionation
! Flux pairs: C4H8(748), C4H7(191); CH3(18),

   to ! Template reaction: Disproportionation
! Flux pairs: S(777), S(252); C2H5O2(47), C2H6O2(48); 
! Estimated from node Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_Ext-6R!H-R
C2H5O2(47)+S(777)=C2H6O2(48)+S(252)                 2.186870e-01 2.659     0.000    

replacing kinetics for C2H5O(49) + C2H5O2(47) <=> CH3CHO(35) + C2H6O2(48)
 from ! Template reaction: Disproportionation
! Flux pairs: C2H5O2(47), C2H6O2(48); C2H5O(49), CH3CHO(35); 
! Estimated from node Root_Ext-1R!H-R
! Multiplied by reaction path degeneracy 2.0
C2H5O(49)+C2H5O2(47)=CH3CHO(35)+C2H6O2(48)          1.889016e+10 0.509     0.000    

   to ! Template reaction: Disproportionation
! Flux pairs: C2H5O2(47), C2H6O2(48); C2H5O(49), CH3CHO(35); 
! Fitted to 50 data points; dA = *|/ 1.05468, dn = +|- 0.00699439, dEa = +|- 0.0380632 kJ/molMatched reaction 186 C2H5O2(47) + C2H5O(49) <=> C2H6O2(48) + CH3CHO(35)-3 in Disproportionation/training
! This reaction matched rate rule [Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-

   to ! Template reaction: Disproportionation
! Flux pairs: S(186), S(187); NC3H7(92), C3H6(12); 
! Estimated from node Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R
NC3H7(92)+S(186)=C3H6(12)+S(187)                    1.115240e+16 -0.957    0.000    

replacing kinetics for NC3H7(92) + S(223) <=> C3H6(12) + S(187)
 from ! Template reaction: Disproportionation
! Flux pairs: S(223), S(187); NC3H7(92), C3H6(12); 
! Estimated from node Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Sp-6C-4C_Ext-6C-R
! Multiplied by reaction path degeneracy 2.0
NC3H7(92)+S(223)=C3H6(12)+S(187)                    2.900000e+12 0.000     0.000    

   to ! Template reaction: Disproportionation
! Flux pairs: S(223), S(187); NC3H7(92), C3H6(12); 
! Estimated from node Root_1R!H->C_Ext-4R-R_N-4R->O_N-5R!H->S_N-Sp-5CCCCCHHNNOOOSS#4CCCCCCHHHNNNOOOSSS_Ext-1C-R_6R!H->C_Sp-6C-1C_4CHNS->C_5CO->C_Sp-5C-4C_Ext-5C-R
NC3H7(92

   to ! Template reaction: Disproportionation
! Flux pairs: S(223), S(187); C4H8(748), C4H7(190); 
! Estimated from node Root_1R!H->C_Ext-4R-R_N-4R->O_N-5R!H->S_Ext-2R!H-R
C4H8(748)+S(223)=C4H7(190)+S(187)                   8.722800e+09 0.657     0.000    

replacing kinetics for C4H7(191) + S(186) <=> C4H6(194) + S(187)
 from ! Template reaction: Disproportionation
! Flux pairs: S(186), S(187); C4H7(191), C4H6(194); 
! Estimated from node Root_Ext-1R!H-R_4R->O_Ext-4O-R_Sp-5R!H-1R!H_Ext-5R!H-R
! Multiplied by reaction path degeneracy 2.0
C4H7(191)+S(186)=C4H6(194)+S(187)                   2.000000e+10 -0.000    0.000    

   to ! Template reaction: Disproportionation
! Flux pairs: S(186), S(187); C4H7(191), C4H6(194); 
! Estimated from node Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_Ext-6R!H-R
C4H7(191)+S(186)=C4H6(194)+S(187)                   2.186870e-01 2.659     0.000    

replacing kinetics for C4H7(191) + S(223) <=> C4H6(194) + S(187)
 from ! Template reaction: Disproportionat

In [10]:
# Load the new kinetics library
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
kinetics_lib = os.path.join(DFT_DIR, 'kinetics', 'kinetics')
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.load_libraries(kinetics_lib)
print(f'{len(ark_kinetics_database.libraries[""].entries)} entries loaded')

105 entries loaded


In [11]:
# load the uncertainty matrix
covariance_file = '/work/westgroup/harris.se/autoscience/autoscience/uncertainty/butane_covariance.pickle'
with open(covariance_file, 'rb') as handle:
    Sigma_k = pickle.load(handle)

uncertainty_cutoff = 3.0

change_list = []
for i in range(0, len(reaction_list)):
    if Sigma_k[i, i] >= uncertainty_cutoff:
        change_list.append(i)



In [ ]:
# load Matt's uncorrelated uncertainty
uncorrelated_uncertainties = np.load('/work/westgroup/harris.se/autoscience/autoscience/uncertainty/uncorrelated_rxn_uncertainties.npy')
print(len(uncorrelated_uncertainties))

uncertainty_cutoff = 3.0
change_list = []
for i in range(0, len(reaction_list)):
    if uncorrelated_uncertainties[i] >= uncertainty_cutoff:
        change_list.append(i)



In [12]:
# stitch each of the library reactions into the mechanism (depends on library index matching indexing in chemkin file)
# change_list = [915, 749, 419, 1814, 1103, 581, 464, 720, 722, 1658, 427, 1721]  # 5.0 cutoff
# change_list = [324, 748, 1287, 915, 213, 749, 1288, 370, 419, 371, 1814, 420,  # 3.0 cutoff
#     1736, 1665, 1685, 1103, 1766, 1714, 1289, 418, 1290, 574, 584, 725, 581,
#     464, 720, 722, 1328, 1773, 1658, 1114, 1688, 739, 582, 1342, 1295, 1026,
#     843, 1111, 427, 1721, 1305, 721, 737, 1734, 719, 538,
#     877, 1304, 1348, 1109, 423, 1181, 740]

# change_list_fs = [584, 213, 1111, 582, 733, 732, 1685, 1736, 748, 464, 1766, 725, 552, 481, 532, 542, 754, 588, 574, 595, 477,
#                   1467, 749, 254, 843, 533, 1665, 478, 1814, 585, 535, 753, 420, 534, 297, 419, 722, 1060, 720, 538, 581, 1022,
#                   844, 217, 424, 551, 1114]



total_changed = 0
for key in ark_kinetics_database.libraries[''].entries.keys():
    entry = ark_kinetics_database.libraries[''].entries[key]
    index = entry.index
#     print(index, '\t', entry.label, '\t', reaction_list[index])
    for i in range(0, len(reaction_list)):
        if entry.item.is_isomorphic(reaction_list[i]):
            rmg_rxn_index = i
            break
    else:  
        print(f'could not match reaction index {index}')
        continue
  
    
#     # change every possible reaction
#     print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
#     reaction_list[rmg_rxn_index].kinetics = entry.data
    
    if rmg_rxn_index in change_list:
        print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
        reaction_list[rmg_rxn_index].kinetics = entry.data
        total_changed += 1

#     if rmg_rxn_index in change_list or rmg_rxn_index in change_list_fs:
#         print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
#         reaction_list[rmg_rxn_index].kinetics = entry.data
#         total_changed += 1
print(total_changed, 'changed')

changing reaction 213 HO2(16) + C2H5(33) <=> H2O2(17) + C2H4(11)
changing reaction 324 S(229) <=> S(225)
changing reaction 370 CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)
changing reaction 371 CH3CHO(35) + PC4H9(182) <=> CH2CHO(21) + butane(1)
changing reaction 418 HO2(16) + CH3CO(20) <=> H2O2(17) + CH2CO(24)
changing reaction 419 HO2(16) + CH2CHO(21) <=> H2O2(17) + CH2CO(24)
changing reaction 420 HO2(16) + SC4H9(183) <=> H2O2(17) + C4H8(189)
changing reaction 427 CH2CHO(21) + CH2CHO(21) <=> CH2CO(24) + CH3CHO(35)
changing reaction 574 CH3O(31) + C3H5-A(94) <=> CH2O(9) + C3H6(12)
changing reaction 581 CH2CHO(21) + C3H5-A(94) <=> CH2CO(24) + C3H6(12)
changing reaction 720 CH2CHO(21) + C4H7(190) <=> CH2CO(24) + C4H8(189)
changing reaction 722 CH2CHO(21) + C4H7(190) <=> CH2CO(24) + C4H8(188)
changing reaction 725 CH3O(31) + C4H7(190) <=> CH2O(9) + C4H8(189)
changing reaction 739 CH2CHO(21) + C4H8(188) <=> CH3CHO(35) + C4H7(190)
changing reaction 748 HCO(19) + C4H8(189) <=> CH2O(9) 

# 

In [13]:
# Load the new thermo library
thermo_lib = os.path.join(DFT_DIR, 'thermo', 'thermo')
ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
ark_thermo_database.load_libraries(thermo_lib)
print(f'{len(ark_thermo_database.libraries["harris_butane"].entries)} entries')

110 entries


In [14]:
# stitch each of the library thermos into the mechanism

plot_index = [96]
thermo_change_list = [87, 90, 109, 99, 98, 80, 100, 97, 81, 91, 82, 93]  # 3.0 cutoff
thermo_change_list_fs = [97, 98, 99, 81, 100, 87, 109, 80, 90, 93, 91, 82]  # 3.0 cutoff, flame speeds these are the same

# thermo_change_list = [63, 4, 66, 57, 61, 97, 62, 59, 83, 64, 74, 60, 98, 39, 45, 37, 47, 70, 73, 38, 72,  # 1.0 cutoff, large molecules only
#                       58, 85, 69, 52, 65, 56, 99, 81, 92, 67, 51, 101, 68, 95, 54, 100, 105, 103, 42, 87,
#                       55, 43, 84, 46, 53, 109, 104, 107, 77, 102, 78, 71, 88, 76, 90, 93, 44, 89, 86, 108, 106, 82, 75]

# thermo_change_list = [97, 83, 98, 85, 99, 81, 92, 101, 100, 105, 103, 87, 84, 109, 107, 102, 88, 90, 93, 89, 86, 108, 106, 82]  # 1.5 cutoff, MW>48
# thermo_change_list = [97, 98, 85, 99, 81, 101, 100, 105, 87, 109, 90, 93, 82]  # 2.5 cutoff, large molecules only


# thermo_change_list = [85,  # 2.0 cutoff
#  87, 84, 105, 103, 90, 102, 109, 99, 107, 83, 92, 98, 80, 100, 97, 88, 81,
#  101, 106, 86, 108, 91, 82, 96, 93, 89]
sp_changed = 0
for key in ark_thermo_database.libraries['harris_butane'].entries.keys():
    entry = ark_thermo_database.libraries['harris_butane'].entries[key]
#     print(entry)

    for i, species in enumerate(species_list):
#         if entry.item.is_isomorphic(species.molecule[0]):
        if entry.item.smiles == species.smiles:
            rmg_species_index = i
            break
    else:
        entry_sp = rmgpy.species.Species(smiles=entry.item.smiles)
        for j, species in enumerate(species_list):
            if entry_sp.is_isomorphic(species.molecule[0]):
                rmg_species_index = i
                'matched'
                break
        else:
#             print(f'could not match species index {entry}')
            continue
            raise ValueError (f'could not match species index {entry}')

#     # only change GAV entries
#     if 'library' not in species_list[rmg_species_index].thermo.comment.lower():
#         print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
# #         try:
# #             display(species_list[rmg_species_index])
# #         except:
# #             pass
# #         plot_thermos([species_list[rmg_species_index], entry.data], ['RMG', 'DFT'])
#         species_list[rmg_species_index].thermo = entry.data
# #         print(rmg_species_index)
#     if rmg_species_index in thermo_change_list:
#         print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
#         species_list[rmg_species_index].thermo = entry.data
        
        
    if rmg_species_index in thermo_change_list or rmg_species_index in thermo_change_list_fs:
        print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
        species_list[rmg_species_index].thermo = entry.data
        
        sp_changed += 1
    
print(sp_changed, 'changed')   
        
#     # change everything
#     print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
#     if rmg_species_index == 85:
# #         plot_thermos([species_list[rmg_species_index], entry.data], ['RMG', 'DFT'])
#         species_list[rmg_species_index].thermo = entry.data

    
    # only change GAV
    
    
#     if rmg_species_index in plot_index:
#         plot_thermos([species_list[rmg_species_index], entry.data], ['RMG', 'DFT'])
#         print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
#         species_list[rmg_species_index].thermo = entry.data

#     break
#     print(species_list[rmg_species_index])
#     print(f'Changing reaction {rmg_rxn_index}')
#     if rmg_rxn_index == 288:
#         reaction_list[rmg_rxn_index].kinetics = entry.data
# #     break
#     # compare the kinetics
#     plot_kinetics([reaction_list[rmg_rxn_index], entry.data], ['RMG', 'DFT'])
    

Changing estimated thermo 80: C2H4O(703)
Changing estimated thermo 81: C4H8(748)
Changing estimated thermo 82: C4H8O(766)
Changing estimated thermo 87: S(911)
Changing estimated thermo 90: S(928)
Changing estimated thermo 91: CH2O2(988)
Changing estimated thermo 93: C4H6(2534)
Changing estimated thermo 97: S(4243)
Changing estimated thermo 98: S(4244)
Changing estimated thermo 99: C4H7(4343)
Changing estimated thermo 100: S(4407)
Changing estimated thermo 109: S(9510)
12 changed


In [15]:
# custom save
chemkin_file = os.path.join('/work/westgroup/harris.se/autoscience/autoscience/butane/models/modifications', f'cutoff3_retrained_20221109.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={chemkin_file[:-4]}.cti'])
# os.remove(chemkin_file)


Wrote CTI mechanism file to '/work/westgroup/harris.se/autoscience/autoscience/butane/models/modifications/cutoff3_retrained_20221109.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.


CompletedProcess(args=['ck2cti', '--input=/work/westgroup/harris.se/autoscience/autoscience/butane/models/modifications/cutoff3_retrained_20221109.inp', '--transport=/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model/tran.dat', '--output=/work/westgroup/harris.se/autoscience/autoscience/butane/models/modifications/cutoff3_retrained_20221109.cti'], returncode=0)

In [ ]:
# save the resulting mechanism
timestamp_str = datetime.datetime.now().isoformat()[0:10]
chemkin_file = os.path.join(new_model_dir, f'chem_{timestamp_str}.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={new_model_dir}/chem_{timestamp_str}.cti'])
# os.remove(chemkin_file)

